# Understanding Different Medical Image Loading Libraries

This notebook compares and contrasts how different Python libraries handle medical image loading, specifically focusing on:
- PyDICOM
- NiBabel
- SimpleITK

We'll explore how each library:
1. Loads image data
2. Handles coordinate systems
3. Manages image orientations
4. Stores metadata

Understanding these differences is crucial for medical image analysis as it helps prevent errors in image processing and ensures correct interpretation of results.


In [98]:
# Ensure we're in the correct directory and set up paths
import os
from pathlib import Path

# Get the notebook's directory and set up paths
notebook_dir = Path(os.getcwd()).resolve()
if notebook_dir.name != 'ch2_imaging':
    # If we're not in the correct directory, try to find and change to it
    if (notebook_dir / 'notebooks' / 'ch2_imaging').exists():
        os.chdir(notebook_dir / 'notebooks' / 'ch2_imaging')
    elif (notebook_dir.parent / 'ch2_imaging').exists():
        os.chdir(notebook_dir.parent / 'ch2_imaging')
    notebook_dir = Path(os.getcwd()).resolve()

# Set up data paths
data_dir = notebook_dir.parent / 'data'
dicom_dir = data_dir / 'example-dicom-structural/dicoms'
nifti_file = data_dir / 'example-dicom-structural/structural.nii.gz'

print(f"Current working directory: {notebook_dir}")
print(f"Data directory: {data_dir}")
print(f"DICOM directory: {dicom_dir}")
print(f"NIfTI file: {nifti_file}")

# Verify paths exist
assert dicom_dir.exists(), f"DICOM directory not found at {dicom_dir}"
assert nifti_file.exists(), f"NIfTI file not found at {nifti_file}"


Current working directory: /Users/justin/Documents/AIMI-tutorial/notebooks/ch2_imaging
Data directory: /Users/justin/Documents/AIMI-tutorial/notebooks/data
DICOM directory: /Users/justin/Documents/AIMI-tutorial/notebooks/data/example-dicom-structural/dicoms
NIfTI file: /Users/justin/Documents/AIMI-tutorial/notebooks/data/example-dicom-structural/structural.nii.gz


In [99]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import nibabel as nib
import SimpleITK as sitk
from pathlib import Path

# Set up the plotting parameters
plt.style.use('seaborn-v0_8')  # Using specific seaborn style version
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 12


# Image Loading and Array Orientation

Each library has its own way of loading and organizing image data. Let's examine these differences:

1. **PyDICOM**:
   - Loads DICOM files with `pixel_array`
   - Array orientation: [row, column]
   - Origin is typically top-left
   - Follows radiological convention

2. **NiBabel**:
   - Loads NIfTI files with get_fdata()
   - Array orientation: [x, y, z] or [t, x, y, z]
   - Uses RAS+ coordinate system (Right, Anterior, Superior)
   - Origin is typically at the back-left-bottom of the head

3. **SimpleITK**:
   - Can load multiple formats (DICOM, NIfTI, etc.)
   - Array orientation: [x, y, z]
   - Follows ITK's coordinate system
   - Provides unified interface for different formats

Let's demonstrate these differences with examples:


# Data Directory Structure

This notebook expects the following directory structure:
```
AIMI-tutorial/
├── notebooks/
│   ├── data/                    # Common data directory for all notebooks
│   │   ├── example-dicom-structural/
│   │   └── ...
│   └── ch2_imaging/            # Current notebook directory
│       └── 02_image_loading_comparison.ipynb
```

The data directory is shared across all notebooks in the repository, making it easier to manage and share datasets between different examples and exercises.


In [100]:
def visualize_array_orientation(img_array, title, axis_labels=None):
    """
    Visualize a 2D image array with its axis orientations.
    
    Parameters:
    -----------
    img_array : numpy.ndarray
        2D image array to visualize
    title : str
        Title of the plot
    axis_labels : tuple
        Labels for x and y axes (default: None)
    """
    if img_array is None:
        print("Error: No image data provided")
        return
        
    if len(img_array.shape) > 2:
        print(f"Warning: Input array has {len(img_array.shape)} dimensions. Using first 2D slice.")
        if len(img_array.shape) == 3:
            img_array = img_array[:, :, 0]
        else:
            img_array = img_array[:, :, 0, 0]
            
    # Create figure with a dark background style
    with plt.style.context('seaborn-darkgrid'):
        fig, ax = plt.subplots(figsize=(12, 12))
        
        # Normalize image data for better visualization
        img_normalized = (img_array - img_array.min()) / (img_array.max() - img_array.min())
        
        # Plot the image with improved contrast
        im = ax.imshow(img_normalized, cmap='gray')
        
        # Add a more informative colorbar
        cbar = plt.colorbar(im)
        cbar.set_label('Normalized Intensity', rotation=270, labelpad=15)
        
        # Add grid with better visibility
        ax.grid(True, linestyle='--', alpha=0.4, color='white')
        
        # Add arrows with improved visibility
        arrow_props = dict(arrowstyle='->', color='red', lw=2.5,
                         mutation_scale=15, shrinkA=0, shrinkB=0)
        
        # Add directional arrows
        ax.annotate('', xy=(img_array.shape[1]-1, img_array.shape[0]/2),
                    xytext=(0, img_array.shape[0]/2), arrowprops=arrow_props)
        ax.annotate('', xy=(img_array.shape[1]/2, img_array.shape[0]-1),
                    xytext=(img_array.shape[1]/2, 0), arrowprops=arrow_props)
        
        # Add axis labels with better positioning and visibility
        if axis_labels:
            ax.text(img_array.shape[1]/2, -img_array.shape[0]*0.05, 
                   axis_labels[0], ha='center', va='center', 
                   color='red', fontsize=12, fontweight='bold')
            ax.text(-img_array.shape[1]*0.05, img_array.shape[0]/2, 
                   axis_labels[1], va='center', ha='center',
                   rotation=90, color='red', fontsize=12, fontweight='bold')
        
        # Add title with better styling
        plt.title(title, pad=20, fontsize=14, fontweight='bold')
        
        # Improve layout
        plt.tight_layout()
        plt.show()


# Example: Loading and Comparing the Same Image with Different Libraries

We'll load the same image using each library and visualize how they handle the data differently. For this example, we'll use both a DICOM image and a NIfTI image to demonstrate the differences in:
1. Array shape and orientation
2. Coordinate systems
3. Metadata handling

First, let's check if we have sample data in our data directory:


In [101]:
# Set up data directory (now under notebooks/data)
notebook_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
data_dir = notebook_dir / 'data'
if not data_dir.exists():
    data_dir.mkdir(parents=True)
    print("Created data directory.")

print("\nAvailable files in data directory:")
files = list(data_dir.glob('*'))
if files:
    for file in files:
        print(f"- {file.name}")
else:
    print("No files found. Please add sample DICOM and NIfTI files to the data directory.")
    print("\nExpected files:")
    print("- sample.dcm (DICOM file)")
    print("- sample.nii.gz (NIfTI file)")
    print("\nNote: You can obtain sample medical images from public datasets such as:")
    print("1. TCIA Collections (https://www.cancerimagingarchive.net)")
    print("2. MedNIST (https://medmnist.com)")
    print("3. IXI Dataset (https://brain-development.org/ixi-dataset)")



Available files in data directory:
- example-dicom-structural


# Loading the Same Image with Different Libraries

Below, we'll demonstrate how to load the same image using each library and show the differences in:
1. How the data is stored in memory
2. The orientation of the arrays
3. How to access metadata
4. How to properly display the images

Note: Make sure you have appropriate sample DICOM and NIfTI files in your data directory. If you don't see them in the output above, you'll need to add them before proceeding.

## Key Differences Summary:

1. **Array Indexing**:
   - PyDICOM: `[row, col]` - follows matrix convention
   - NiBabel: `[x, y, z]` - follows RAS+ convention
   - SimpleITK: `[x, y, z]` - follows ITK convention

2. **Coordinate Systems**:
   - PyDICOM: Patient-relative coordinates from DICOM tags
   - NiBabel: RAS+ coordinate system
   - SimpleITK: ITK coordinate system with physical space information

3. **Metadata Handling**:
   - PyDICOM: Complete DICOM metadata in dictionary format
   - NiBabel: Header information in structured format
   - SimpleITK: Unified metadata interface for all formats

Let's see these differences in action:


In [102]:
def load_and_compare_image(dicom_path, nifti_path=None, slice_idx=None):
    """
    Load and compare medical images using different libraries.
    
    Parameters:
    -----------
    dicom_path : str or Path
        Path to the DICOM file
    nifti_path : str or Path, optional
        Path to the NIfTI file (if None, only DICOM will be loaded)
    slice_idx : int, optional
        Specific slice index to visualize for 3D data (if None, middle slice is used)
    """
    results = {}
    
    # Load with PyDICOM
    try:
        dcm = pydicom.dcmread(str(dicom_path))
        dcm_array = dcm.pixel_array
        results['pydicom'] = {
            'array': dcm_array,
            'metadata': {
                'shape': dcm_array.shape,
                'dtype': str(dcm_array.dtype),
                'PatientPosition': getattr(dcm, 'PatientPosition', 'N/A'),
                'ImageOrientation': getattr(dcm, 'ImageOrientationPatient', 'N/A'),
                'PixelSpacing': getattr(dcm, 'PixelSpacing', 'N/A'),
                'SliceThickness': getattr(dcm, 'SliceThickness', 'N/A')
            }
        }
        print("\nPyDICOM loading:")
        for key, value in results['pydicom']['metadata'].items():
            print(f"{key}: {value}")
        print("-" * 50)
        
        # Visualize PyDICOM array
        visualize_array_orientation(dcm_array, "PyDICOM Array Orientation", 
                                 ("Column (Width)", "Row (Height)"))
    except Exception as e:
        print(f"Error loading DICOM: {e}")
    
    if nifti_path is not None:
        # Load with NiBabel
        try:
            nii = nib.load(str(nifti_path))
            nii_array = nii.get_fdata()
            results['nibabel'] = {
                'array': nii_array,
                'metadata': {
                    'shape': nii_array.shape,
                    'dtype': str(nii_array.dtype),
                    'affine': nii.affine,
                    'header': dict(nii.header)
                }
            }
            print("\nNiBabel loading:")
            print(f"Shape: {nii_array.shape}")
            print(f"Data type: {nii_array.dtype}")
            print(f"Affine matrix:\n{nii.affine}")
            print("-" * 50)
            
            # Get appropriate slice
            if len(nii_array.shape) == 3:
                if slice_idx is None:
                    slice_idx = nii_array.shape[2]//2
                middle_slice = nii_array[:, :, slice_idx]
            else:
                middle_slice = nii_array
            visualize_array_orientation(middle_slice, f"NiBabel Array Orientation (Slice {slice_idx})",
                                     ("R -> L", "P -> A"))
        except Exception as e:
            print(f"Error loading NIfTI: {e}")
        
        # Load with SimpleITK
        try:
            sitk_img = sitk.ReadImage(str(nifti_path))
            sitk_array = sitk.GetArrayFromImage(sitk_img)
            results['sitk'] = {
                'array': sitk_array,
                'metadata': {
                    'shape': sitk_array.shape,
                    'dtype': str(sitk_array.dtype),
                    'origin': sitk_img.GetOrigin(),
                    'spacing': sitk_img.GetSpacing(),
                    'direction': sitk_img.GetDirection()
                }
            }
            print("\nSimpleITK loading:")
            for key, value in results['sitk']['metadata'].items():
                print(f"{key}: {value}")
            print("-" * 50)
            
            # Get appropriate slice
            if len(sitk_array.shape) == 3:
                if slice_idx is None:
                    slice_idx = sitk_array.shape[0]//2
                middle_slice = sitk_array[slice_idx, :, :]
            else:
                middle_slice = sitk_array
            visualize_array_orientation(middle_slice, f"SimpleITK Array Orientation (Slice {slice_idx})",
                                     ("Column", "Row"))
        except Exception as e:
            print(f"Error loading with SimpleITK: {e}")
    
    return results


# Summary of Key Differences

Let's summarize the key differences between these libraries:

1. **Array Orientation and Shape**:
   - PyDICOM: Native 2D array with [row, column] indexing
   - NiBabel: Uses RAS+ coordinate system, typically [x, y, z] for 3D data
   - SimpleITK: Array indexing is [z, y, x] when converted to NumPy array

2. **Coordinate Systems**:
   - PyDICOM: Uses DICOM patient coordinate system
   - NiBabel: RAS+ (Right, Anterior, Superior)
   - SimpleITK: ITK coordinate system with physical space information

3. **Metadata Access**:
   - PyDICOM: Direct access to DICOM tags
   - NiBabel: Structured header with affine transformation
   - SimpleITK: Unified interface for spatial information


4. **Data Types and Multi-frame Support**:
   - PyDICOM: Preserves original DICOM data type, handles single and multi-frame DICOM
   - NiBabel: Converts to float64 by default, native support for 3D/4D data
   - SimpleITK: Preserves original data type when possible, unified handling of 2D-5D data

These differences are important to consider when:
- Converting between different formats
- Ensuring correct orientation in visualization
- Performing spatial operations
- Working with multi-modal image registration
- Implementing processing pipelines

To use this notebook effectively:
1. Place your DICOM and NIfTI files in the `data` directory
2. Update the file paths in the code below
3. Run the comparison to see the differences in action


In [103]:
# Example usage (update these paths with your actual file paths)
data_dir = notebook_dir.parent.parent / 'notebooks' / 'data'  # Use the common data directory
dicom_file = data_dir / "sample.dcm"  # Replace with your DICOM file
nifti_file = data_dir / "sample.nii.gz"  # Replace with your NIfTI file

if dicom_file.exists() and nifti_file.exists():
    load_and_compare_image(dicom_file, nifti_file)
else:
    print("Please add sample DICOM and NIfTI files to the data directory to run this comparison.")


Please add sample DICOM and NIfTI files to the data directory to run this comparison.


In [104]:
# Example usage with the available DICOM directory
data_dir = notebook_dir.parent.parent / 'notebooks' / 'data'  # Use the common data directory
dicom_dir = data_dir / "example-dicom-structural"
if dicom_dir.exists():
    # Find all DICOM files in the directory
    dicom_files = list(dicom_dir.glob('*.dcm'))
    if dicom_files:
        print(f"Found {len(dicom_files)} DICOM files in {dicom_dir}")
        # Use the first DICOM file for demonstration
        dicom_file = dicom_files[0]
        print(f"Using {dicom_file.name} for demonstration")
        
        # For this example, we'll only demonstrate DICOM loading since we don't have a NIfTI file
        try:
            # Load and display DICOM only
            dcm = pydicom.dcmread(str(dicom_file))
            dcm_array = dcm.pixel_array
            print("\nPyDICOM loading:")
            print(f"Array shape: {dcm_array.shape}")
            print(f"Data type: {dcm_array.dtype}")
            print(f"Array orientation: [row, column]")
            print("\nExample metadata:")
            print(f"Patient Position: {getattr(dcm, 'PatientPosition', 'N/A')}")
            print(f"Image Orientation: {getattr(dcm, 'ImageOrientationPatient', 'N/A')}")
            print("-" * 50)
            
            # Visualize PyDICOM array
            visualize_array_orientation(dcm_array, "PyDICOM Array Orientation", 
                                     ("Column (Width)", "Row (Height)"))
        except Exception as e:
            print(f"Error loading DICOM: {e}")
    else:
        print("No DICOM files found in the example-dicom-structural directory.")
else:
    print("Please add sample DICOM and NIfTI files to the data directory to run this comparison.")


Please add sample DICOM and NIfTI files to the data directory to run this comparison.


# Converting DICOM to NIfTI

To see the full comparison between different libraries, you might want to convert your DICOM files to NIfTI format. Here are a few ways to do this:

1. Using SimpleITK:
```python
# Read DICOM series
reader = sitk.ImageSeriesReader()
dicom_names = reader.GetGDCMSeriesFileNames(str(dicom_dir))
reader.SetFileNames(dicom_names)
image = reader.Execute()

# Write as NIfTI
sitk.WriteImage(image, "output.nii.gz")
```

2. Using dcm2niix (command line tool):
```bash
dcm2niix -z y -f %p_%s -o output_dir input_dicom_dir
```

3. Using dicom2nifti (Python package):
```python
import dicom2nifti
dicom2nifti.convert_directory(str(dicom_dir), "output_dir")
```

Once you have both DICOM and NIfTI versions of the same image, you can run the full comparison to see how each library handles the data differently.


In [105]:
# Test loading with actual data
data_dir = notebook_dir.parent / 'data'
dicom_dir = data_dir / 'example-dicom-structural/dicoms'
nifti_file = data_dir / 'example-dicom-structural/structural.nii.gz'

print(f"DICOM directory exists: {dicom_dir.exists()}")
print(f"NIfTI file exists: {nifti_file.exists()}")

if dicom_dir.exists() and nifti_file.exists():
    # Get first DICOM file
    dicom_files = sorted(list(dicom_dir.glob('*.dcm')))
    if dicom_files:
        dicom_file = dicom_files[0]
        print(f"\nUsing DICOM file: {dicom_file.name}")
        
        # Load and compare
        results = load_and_compare_image(dicom_file, nifti_file)
        
        # Print additional comparison information
        if 'pydicom' in results and 'sitk' in results:
            print("\nShape comparison:")
            print(f"PyDICOM shape: {results['pydicom']['metadata']['shape']}")
            print(f"SimpleITK shape: {results['sitk']['metadata']['shape']}")
            if 'nibabel' in results:
                print(f"NiBabel shape: {results['nibabel']['metadata']['shape']}")
    else:
        print("No DICOM files found in the directory.")


DICOM directory exists: False
NIfTI file exists: False


In [106]:
# Example usage with our actual data
data_dir = notebook_dir.parent / 'data'  # Use the common data directory
dicom_dir = data_dir / "example-dicom-structural/dicoms"
nifti_file = data_dir / "example-dicom-structural/structural.nii.gz"

if dicom_dir.exists() and nifti_file.exists():
    # Find all DICOM files in the directory
    dicom_files = list(dicom_dir.glob('*.dcm'))
    if dicom_files:
        print(f"Found {len(dicom_files)} DICOM files in {dicom_dir}")
        # Use the first DICOM file for demonstration
        dicom_file = dicom_files[0]
        print(f"Using {dicom_file.name} for demonstration")
        
        # Now we can demonstrate the full comparison
        load_and_compare_image(dicom_file, nifti_file)
    else:
        print("No DICOM files found in the directory.")
else:
    print("Please ensure both DICOM and NIfTI data are available in the data directory.")


Please ensure both DICOM and NIfTI data are available in the data directory.


In [107]:
def load_and_compare_image(dicom_path, nifti_path):
    """
    Load the same image using different libraries and compare their representations.
    
    Parameters:
    -----------
    dicom_path : str or Path
        Path to the DICOM file
    nifti_path : str or Path
        Path to the NIfTI file
    """
    # Load with PyDICOM
    try:
        dcm = pydicom.dcmread(str(dicom_path))
        dcm_array = dcm.pixel_array
        print("PyDICOM loading:")
        print(f"Array shape: {dcm_array.shape}")
        print(f"Data type: {dcm_array.dtype}")
        print(f"Array orientation: [row, column]")
        print("\nExample metadata:")
        print(f"Patient Position: {getattr(dcm, 'PatientPosition', 'N/A')}")
        print(f"Image Orientation: {getattr(dcm, 'ImageOrientationPatient', 'N/A')}")
        print("-" * 50)
        
        # Visualize PyDICOM array
        visualize_array_orientation(dcm_array, "PyDICOM Array Orientation", 
                                 ("Column (Width)", "Row (Height)"))
    except Exception as e:
        print(f"Error loading DICOM: {e}")
    
    # Load with NiBabel
    try:
        nii = nib.load(str(nifti_path))
        nii_array = nii.get_fdata()
        print("\nNiBabel loading:")
        print(f"Array shape: {nii_array.shape}")
        print(f"Data type: {nii_array.dtype}")
        print(f"Affine matrix:\n{nii.affine}")
        print(f"Array orientation: RAS+ [Right, Anterior, Superior]")
        print("-" * 50)
        
        # Visualize NiBabel array (middle slice if 3D)
        if len(nii_array.shape) == 3:
            middle_slice = nii_array[:, :, nii_array.shape[2]//2]
        else:
            middle_slice = nii_array
        visualize_array_orientation(middle_slice, "NiBabel Array Orientation",
                                 ("R -> L", "P -> A"))
    except Exception as e:
        print(f"Error loading NIfTI: {e}")
    
    # Load with SimpleITK
    try:
        sitk_img = sitk.ReadImage(str(nifti_path))  # Can read both DICOM and NIfTI
        sitk_array = sitk.GetArrayFromImage(sitk_img)
        print("\nSimpleITK loading:")
        print(f"Array shape: {sitk_array.shape}")
        print(f"Data type: {sitk_array.dtype}")
        print(f"Origin: {sitk_img.GetOrigin()}")
        print(f"Spacing: {sitk_img.GetSpacing()}")
        print(f"Direction: {sitk_img.GetDirection()}")
        print("-" * 50)
        
        # Visualize SimpleITK array (middle slice if 3D)
        if len(sitk_array.shape) == 3:
            middle_slice = sitk_array[sitk_array.shape[0]//2, :, :]
        else:
            middle_slice = sitk_array
        visualize_array_orientation(middle_slice, "SimpleITK Array Orientation",
                                 ("Column", "Row"))
    except Exception as e:
        print(f"Error loading with SimpleITK: {e}")
